In [2]:
import openai  # openai==0.27.8

.py文件需要指定api_key和api_base

In [6]:
from dotenv import load_dotenv
import os
import json

load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_base = os.getenv("OPENAI_API_BASE")

一些官方资料：
1. https://platform.openai.com/examples
2. https://github.com/openai/openai-cookbook/tree/main/examples

最简单的模板

In [12]:
prompt = '大语言模型有哪些应用场景？'

response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "user", "content": prompt}
    ],
)

print(response.choices[0].message.content)

大语言模型可以应用于以下场景：

1. 机器翻译：大语言模型可以用于改进机器翻译系统，提高翻译质量和流畅度。

2. 文本生成：大语言模型可以用于生成文章、新闻、故事等文本内容，可以用于自动写作、创作辅助等领域。

3. 问答系统：大语言模型可以用于构建智能问答系统，能够回答用户提出的问题，并提供相关的信息和解决方案。

4. 语音识别：大语言模型可以用于提高语音识别系统的准确性和流畅度，使得语音识别结果更加自然和准确。

5. 智能助理：大语言模型可以用于构建智能助理，能够理解用户的指令和需求，并提供相应的服务和建议。

6. 自动摘要：大语言模型可以用于自动摘要生成，能够从大量文本中提取关键信息，生成简洁准确的摘要。

7. 情感分析：大语言模型可以用于情感分析，能够识别文本中的情感倾向，帮助企业了解用户的情感反馈和需求。

8. 聊天机器人：大语言模型可以用于构建聊天机器人，能够进行自然语言对话，提供娱乐、咨询、客服等服务。

总之，大语言模型在自然语言处理领域有广泛的应用，可以提供自动化、智能化的文本处理和生成能力。


提升回答效果的技巧：
1. 指定模型身份
2. 指定回答格式

In [11]:
prompt = '''
假设你是一名金融专家。
请将以下金融相关的问题划分为若干个子问题, 子问题数量不超过5个：

格式样例：
问题：问题
子问题：1.子问题1 2.子问题2 3.子问题3

问题：{question}
子问题：
'''

response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "user", "content": prompt.format(question='大语言模型有哪些应用场景？')}
    ],
)

print(response.choices[0].message.content)

1. 金融领域中的大语言模型应用场景有哪些？
2. 大语言模型在金融领域中的具体应用案例有哪些？
3. 大语言模型在金融领域中的优势和局限性是什么？
4. 大语言模型在金融领域中的发展趋势是什么？
5. 大语言模型在金融领域中的应用所面临的风险和挑战有哪些？


提升回答效果的技巧：

3. 提供示例回答（少样本）

In [14]:
prompt = '''
You will be provided with a tweet, and your task is to classify its sentiment as positive, neutral, or negative.
Here are some examples of tweets and their corresponding sentiment labels:

Tweet: I love the music
Sentiment: Positive

Tweet: I hate the rain
Sentiment: Negative

Tweet: I'm at the beach
Sentiment: Neutral

Here is the tweet to be classified:
{tweet}
'''

response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "user", "content": prompt.format(tweet='"I loved the new Batman movie!"')}
    ],
  temperature=0.0,  # 测评任务需要设置随机性为0
)

print(response.choices[0].message.content)

Sentiment: Positive


Function calling：输出结构化数据，适用于信息抽取功能

In [11]:
prompt = '''
抽取下面这段话的关系三元组：

{context}
'''

context = '''
Black-on-black ware is a 20th- and 21st-century pottery tradition developed by the Puebloan Native American ceramic artists in Northern New Mexico. 
Traditional reduction-fired blackware has been made for centuries by pueblo artists. 
Black-on-black ware of the past century is produced with a smooth surface, with the designs applied through selective burnishing or the application of refractory slip. 
Another style involves carving or incising designs and selectively polishing the raised areas. 
For generations several families from Kha'po Owingeh and P'ohwhóge Owingeh pueblos have been making black-on-black ware with the techniques passed down from matriarch potters. 
Artists from other pueblos have also produced black-on-black ware. Several contemporary artists have created works honoring the pottery of their ancestors.
'''

response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[{"role": "user", "content": prompt.format(context=context)}],
    functions=[
        {
            "name": "get_relation_triple",
            "description": "extract relation triple from text",
            "parameters": {  # 定义输出json的schema
                "type": "object",  # 最外层必须是object
                "properties": {  # 定义object的键值对
                    "info": {  # 定义键的名称
                        "type": "array",  # 定义值的类型
                        "items": {  # 定义array类型的元素
                            "type": "object",  # array的元素类型是object
                            "properties": {
                                "subject": { 
                                    "type": "string",
                                    "description": "subject of triple"  # 解释每个键的含义
                                },
                                "predicate": { 
                                    "type": "string",
                                    "description": "predicate of triple"
                                },
                                "object": { 
                                    "type": "string",
                                    "description": "object of triple"
                                },
                            },
                            'required': ['subject', 'predicate', 'object'],  # 定义必须包含的键
                        },
                    }
                },
                "required": ["info"],  # 定义必须包含的键
            },
        }
    ],
    function_call="auto",
)

print(json.dumps(
    json.loads(response.choices[0].message.function_call.arguments), 
    indent=4, 
    ensure_ascii=False
))

{
    "info": [
        {
            "subject": "Black-on-black ware",
            "predicate": "is",
            "object": "pottery tradition"
        },
        {
            "subject": "Black-on-black ware",
            "predicate": "developed by",
            "object": "Puebloan Native American ceramic artists"
        },
        {
            "subject": "Black-on-black ware",
            "predicate": "developed in",
            "object": "Northern New Mexico"
        },
        {
            "subject": "Traditional reduction-fired blackware",
            "predicate": "made by",
            "object": "pueblo artists"
        },
        {
            "subject": "Black-on-black ware",
            "predicate": "produced with",
            "object": "smooth surface"
        },
        {
            "subject": "Black-on-black ware",
            "predicate": "designs applied through",
            "object": "selective burnishing or application of refractory slip"
        },
        {
   